In [6]:
import os
import csv
import numpy as np
import tensorflow as tf
from itertools import product
from tensorflow_model_optimization.quantization.keras import vitis_quantize
import cv2
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform

from keras.utils.vis_utils import model_to_dot
import keras.backend as K

#from keras_flops import get_flops

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('Training data: {}, {}'.format(x_train.shape, y_train.shape))
print('Test data: {}, {}'.format(x_test.shape, y_test.shape))
x_train = x_train[0:60]
y_train = y_train[0:60]
y_train = np.resize(y_train,[60,2])
x_test = x_test[0:10]
y_test = y_test[0:10]
y_test = np.resize(y_test,[10,2])
x_train, x_test = x_train / 255.0, x_test / 255.0
#x_train = np.expand_dims(x_train, axis=3)
#x_test = np.expand_dims(x_test, axis=3)
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X
#def ResNet50(input_shape = (input_x, input_y, 3), classes = 2):   
def ResNet50(input_x, input_y, classes = 2):
    # Define the input as a tensor with shape input_shape
    #X_input = Input(input_shape)
    X_input = Input(input_shape=(input_x, input_y, 3))

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model
input_horizontal =range(64,256,32)
input_vertical =range(64,256,32)
for input_x, input_y in product(input_horizontal, input_vertical):
    x_train = np.resize(x_train, [60,input_x,input_y,3])
    x_test = np.resize(x_test, [10,input_x,input_y,3])  

    model = ResNet50(input_shape=(input_x, input_y,3), classes = 2)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs = 1)
    model.summary()
    #Total_params = round(model.count_params()/10 ** 6,2)
    #FLOPs = round(get_flops(model, batch_size=1)/10 ** 9,2)
    #print(Total_params,FLOPs)
 

    model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = x_test[1:8], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('tf2_mnist_classifier_quantized.h5')
    !vai_c_tensorflow2 \
            --model ./tf2_mnist_classifier_quantized.h5 \
            --arch ./arch.json \
            --output_dir ./xmodel_resnet50 \
            --net_name deploy
    

    os.rename("./xmodel_resnet50/deploy.xmodel","./xmodel_resnet50/mnist_classifier_resnet50_{input_x}_{input_y}.xmodel".format(input_x=input_x,input_y=input_y))

Training data: (60000, 28, 28), (60000,)
Test data: (10000, 28, 28), (10000,)


TypeError: ResNet50() got an unexpected keyword argument 'input_shape'

In [26]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('Training data: {}, {}'.format(x_train.shape, y_train.shape))
print('Test data: {}, {}'.format(x_test.shape, y_test.shape))
x_train = x_train[0:60]
y_train = y_train[0:60]
x_test = x_test[0:10]
y_test = y_test[0:10]

x_train, x_test = x_train / 255.0, x_test / 255.0
#x_train = np.expand_dims(x_train, axis=3)
#x_test = np.expand_dims(x_test, axis=3)
print(x_train.shape)
x_train = np.resize(x_train, [60,input_x,input_y,3])
print(x_train.shape)

Training data: (60000, 28, 28), (60000,)
Test data: (10000, 28, 28), (10000,)
(60, 28, 28)
(60, 32, 64, 3)


In [10]:
from keras.models import Model
from keras.layers import Flatten, Conv2D, Input, MaxPooling2D, AveragePooling2D, BatchNormalization, add
from keras.layers import ZeroPadding2D, Activation, Dense
from tensorflow_model_optimization.quantization.keras import vitis_quantize
import numpy as np
import os
def conv_bn(x, nb_filters, kernel_size, padding="same", strides=(1, 1), name=None):
    if name is not None:
        bn_name = name + "_bn"
        conv_name = name + "_conv"
    else:
        bn_name = None
        conv_name = None
    x = Conv2D(nb_filters, kernel_size, padding=padding, strides=strides, name=conv_name)(x)

    x = BatchNormalization(axis=-1, name=bn_name)(x)
    x = Activation('relu')(x)

    return x


def bottle_block(input, nb_filters, padding="same", strides=(1, 1), with_conv_shortcut=False):
    k1, k2, k3 = nb_filters
    x = conv_bn(input, k1, (1, 1), padding=padding, strides=strides)
    x = conv_bn(x, k2, (3, 3), padding=padding)
    x = conv_bn(x, k3, (1, 1), padding=padding)
    if with_conv_shortcut:
        shortcut = conv_bn(input, k3, (1, 1), padding=padding, strides=strides)
        x = add([x, shortcut])
    else:
        x = add([x, input])
    return x


def resNet50(height, width, channels, classes):
    input = Input(shape=(height, width, channels))

    x = ZeroPadding2D((3, 3))(input)

    x = conv_bn(x, nb_filters=64, kernel_size=(7, 7), padding="valid", strides=(2, 2))
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)

    x = bottle_block(x, (64, 64, 256), strides=(1, 1), with_conv_shortcut=True)
    x = bottle_block(x, (64, 64, 256), strides=(1, 1), with_conv_shortcut=False)
    x = bottle_block(x, (64, 64, 256), strides=(1, 1), with_conv_shortcut=False)

    x = bottle_block(x, (128, 128, 512), strides=(2, 2), with_conv_shortcut=True)
    x = bottle_block(x, (128, 128, 512), strides=(1, 1), with_conv_shortcut=False)
    x = bottle_block(x, (128, 128, 512), strides=(1, 1), with_conv_shortcut=False)
    x = bottle_block(x, (128, 128, 512), strides=(1, 1), with_conv_shortcut=False)

    x = bottle_block(x, (256, 256, 1024), strides=(2, 2), with_conv_shortcut=True)
    x = bottle_block(x, (256, 256, 1024), strides=(1, 1), with_conv_shortcut=False)
    x = bottle_block(x, (256, 256, 1024), strides=(1, 1), with_conv_shortcut=False)
    x = bottle_block(x, (256, 256, 1024), strides=(1, 1), with_conv_shortcut=False)
    x = bottle_block(x, (256, 256, 1024), strides=(1, 1), with_conv_shortcut=False)
    x = bottle_block(x, (256, 256, 1024), strides=(1, 1), with_conv_shortcut=False)

    x = bottle_block(x, (512, 512, 2048), strides=(2, 2), with_conv_shortcut=True)
    x = bottle_block(x, (512, 512, 2048), strides=(1, 1), with_conv_shortcut=False)
    x = bottle_block(x, (512, 512, 2048), strides=(1, 1), with_conv_shortcut=False)

    x = AveragePooling2D((7, 7))(x)

    x = Flatten()(x)
    output = Dense(classes, activation="softmax")(x)

    model = Model(inputs=input, outputs=output)

    return model

shape_range =range(224,512,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    
    model = resNet50(shape_x, shape_y, 1, 10)
    model.summary()
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, Y_train, epochs=1)
    model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:10], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('tf2_mnist_classifier_quantized.h5')
    !vai_c_tensorflow2 \
    --model ./tf2_mnist_classifier_quantized.h5 \
    --arch ./arch.json \
    --output_dir ./xmodel_resNet50 \
    --net_name deploy
    os.rename("./xmodel_resNet50/deploy.xmodel","./xmodel_resNet50/resNet50_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D (None, 230, 230, 1)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_477 (Conv2D)             (None, 112, 112, 64) 3200        zero_padding2d_9[0][0]           
__________________________________________________________________________________________________
batch_normalization_477 (BatchN (None, 112, 112, 64) 256         conv2d_477[0][0]                 
____________________________________________________________________________________________

2/2 [==============================] - 12s 4s/step - loss: 4.1556 - accuracy: 0.1000
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 10s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 23s 23s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_

2/2 [==============================] - 14s 5s/step - loss: 3.9788 - accuracy: 0.0833
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 11s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 23s 23s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_

2/2 [==============================] - 17s 6s/step - loss: 3.4648 - accuracy: 0.1167
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 11s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 23s 23s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_

2/2 [==============================] - 20s 8s/step - loss: 4.3469 - accuracy: 0.1167
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 10s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 25s 25s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_

2/2 [==============================] - 23s 9s/step - loss: 4.6929 - accuracy: 0.1333
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 11s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 25s 25s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_

2/2 [==============================] - 28s 11s/step - loss: 4.5455 - accuracy: 0.0833
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 10s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 27s 27s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist

2/2 [==============================] - 32s 13s/step - loss: 4.2266 - accuracy: 0.0667
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 11s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 28s 28s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist

2/2 [==============================] - 36s 15s/step - loss: 7.4123 - accuracy: 0.1667
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 12s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 29s 29s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist

2/2 [==============================] - 41s 17s/step - loss: 8.5219 - accuracy: 0.1167
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 11s 1s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 31s 31s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist

In [16]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import  Dense
import numpy as np
import os
from tensorflow_model_optimization.quantization.keras import vitis_quantize
#model = ResNet50(weights=None, include_top=False, input_shape=(200,200,3))
model = ResNet50(weights=None,classes=10)
shape_x = 224
shape_y = shape_x
X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
Y_train=np.random.randint(0,10,size=[60,])
X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
Y_test=np.random.randint(0,10,size=[60,])
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=1)
model.get_weights()[0].dtype
quantizer = vitis_quantize.VitisQuantizer(model)
quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:10], weight_bit=8, activation_bit=8)
quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
quantized_model.save('tf2_mnist_classifier_quantized.h5')
!vai_c_tensorflow2 \
--model ./tf2_mnist_classifier_quantized.h5 \
--arch ./arch.json \
--output_dir ./xmodel_resNet50 \
--net_name deploy
os.rename("./xmodel_resNet50/deploy.xmodel","./xmodel_resNet50/resNet50_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_16[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./tf2_mnist_classifier_quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/tf2_mnist_classifier_quantized.h5
[INFO] keras version: 2.6.0
Traceback (most recent call last):
  File "/opt/vitis_ai/conda/envs/vitis-ai-tensorflow2/bin/xnnc-run", line 33, in <module>
    sys.exit(load_entry_point('xnnc==2.0.0', 'console_scripts', 'xnnc-run')())
  File "/opt/vitis_ai/conda/envs/vitis-ai-tensorflow2/lib/python3.7/site-packages/xnnc/__main__.py", line 49, in main
    runner.normal_run(args)
  File "/opt/vitis_ai/conda/envs/vitis-ai-tensorflow2/lib/python3.7/site-packages/xnnc/runner.py", line 123, in normal_run
    target=target,
  File "/opt/vitis_ai/conda/envs/viti

FileNotFoundError: [Errno 2] No such file or directory: './xmodel_resNet50/deploy.xmodel' -> './xmodel_resNet50/resNet50_224_224.xmodel'

In [13]:
from keras.models import Model
from keras.layers import Input,Dense,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,ZeroPadding2D
from keras.layers import add,Flatten
#from keras.layers.convolutional import Conv2D,MaxPooling2D,AveragePooling2D
from tensorflow.keras.optimizers import SGD
import numpy as np
import os
from tensorflow_model_optimization.quantization.keras import vitis_quantize

def Conv2d_BN(x, nb_filter,kernel_size, strides=(1,1), padding='same',name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
 
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu',name=conv_name)(x)
    x = BatchNormalization(axis=3,name=bn_name)(x)
    return x
 
def Conv_Block(inpt,nb_filter,kernel_size,strides=(1,1), with_conv_shortcut=False):
    x = Conv2d_BN(inpt,nb_filter=nb_filter[0],kernel_size=(1,1),strides=strides,padding='same')
    x = Conv2d_BN(x, nb_filter=nb_filter[1], kernel_size=(3,3), padding='same')
    x = Conv2d_BN(x, nb_filter=nb_filter[2], kernel_size=(1,1), padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt,nb_filter=nb_filter[2],strides=strides,kernel_size=kernel_size)
        x = add([x,shortcut])
        return x
    else:
        x = add([x,inpt])
        return x
 
inpt = Input(shape=(196,196,3))
x = ZeroPadding2D((3,3))(inpt)
x = Conv2d_BN(x,nb_filter=64,kernel_size=(7,7),strides=(2,2),padding='valid')
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
 
x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3),strides=(1,1),with_conv_shortcut=True)
x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3))
x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3))
 
x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)
x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3))
x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3))
x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3))
 
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
 
x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)
x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3))
x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3))
x = AveragePooling2D(pool_size=(7,7))(x)
x = Flatten()(x)
x = Dense(1000,activation='softmax')(x)


shape_range =range(196,436,24)
for shape_x in shape_range:
    
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    
    inpt = Input(shape=(shape_x,shape_y,3))
    
    x = ZeroPadding2D((3,3))(inpt)
    x = Conv2d_BN(x,nb_filter=64,kernel_size=(7,7),strides=(2,2),padding='valid')
    x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

    x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3),strides=(1,1),with_conv_shortcut=True)
    x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3))
    x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3))

    x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)
    x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3))
    x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3))
    x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3))

    x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)
    x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
    x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
    x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
    x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))
    x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3))

    x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True)
    x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3))
    x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3))
    x = AveragePooling2D(pool_size=(7,7))(x)
    x = Flatten()(x)
    x = Dense(10,activation='softmax')(x)
    
    model = Model(inputs=inpt,outputs=x)

    model.summary()
    sgd = SGD(decay=0.0001,momentum=0.9)
    model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])


    #model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, Y_train, epochs=1)
    model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:10], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
    --model ./quantized.h5 \
    --arch ./arch.json \
    --output_dir ./xmodel_resNet50 \
    --net_name deploy
    os.rename("./xmodel_resNet50/deploy.xmodel","./xmodel_resNet50/resNet50_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))



Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 196, 196, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_15 (ZeroPadding2 (None, 202, 202, 3)  0           input_16[0][0]                   
__________________________________________________________________________________________________
conv2d_795 (Conv2D)             (None, 98, 98, 64)   9472        zero_padding2d_15[0][0]          
__________________________________________________________________________________________________
batch_normalization_795 (BatchN (None, 98, 98, 64)   256         conv2d_795[0][0]                 
____________________________________________________________________________________________

2/2 [==============================] - 13s 5s/step - loss: 2.3876 - accuracy: 0.1333
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 24s 3s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 42s 42s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras versi

2/2 [==============================] - 15s 5s/step - loss: 2.3644 - accuracy: 0.0833
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 25s 3s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 41s 41s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras versi

2/2 [==============================] - 17s 7s/step - loss: 2.3679 - accuracy: 0.1000
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 24s 3s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 43s 43s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras versi

2/2 [==============================] - 19s 8s/step - loss: 2.3519 - accuracy: 0.1167
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 25s 3s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 44s 44s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras versi

FileNotFoundError: [Errno 2] No such file or directory: './xmodel_resNet50/deploy.xmodel' -> './xmodel_resNet50/resNet50_268_268.xmodel'